**`Análisis exploratorio de los datos`**: _(Exploratory Data Analysis-EDA)_

Primero importamos las librerias que  vamos a necesitar

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

Cargamos los archivos .csv a los que ya se les aplico las transformaciones

In [4]:
df_amazon_prime = pd.read_csv('../dataset_ETL/amazon_prime_ETL.csv', delimiter = ',',encoding = "utf-8")
df_disney_plus = pd.read_csv('../dataset_ETL/disney_plus_ETL.csv',delimiter= ',', encoding = "utf-8" )
df_hulu = pd.read_csv('../dataset_ETL/hulu_ETL.csv',delimiter= ',', encoding = "utf-8" )
df_netflix = pd.read_csv('../dataset_ETL/netflix_ETL.csv',delimiter= ',', encoding = "utf-8" )
df_ratings = pd.read_parquet('../dataset_ETL/ratings_ETL.csv')#, delimiter=',', encoding= "utf-8")

**Se corre un sample en cada dataframe para ver que tipo de datos hay**

In [5]:
df_amazon_prime.sample(20)

,show_id,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,id,duration_int,duration_type
6195,s6196,movie,to catch a king,clive donner,"robert wagner, teri garr, horst janson",NaN,NaN,1984,16+,"drama, suspense","in 1940, two americans, joe jackson and hannah...",as6196,114,min
6918,s6919,movie,dogged,richard rowntree,"sam saunders, nadia lamin, philip ridout",NaN,NaN,2017,18+,horror,sam is forced to return to the remote island w...,as6919,116,min
5065,s5066,movie,sinbad: where u been?,chuck vinson,sinbad,NaN,NaN,2010,16+,"arts, entertainment, and culture, comedy, spec...","after a multiyear hiatus, venerable funnyman-a...",as5066,90,min
746,s747,movie,the benefactor,andrew renzi,"richard gere, dakota fanning, theo james",NaN,NaN,2016,16+,drama,a newly married couple are forced to navigate ...,as747,93,min
3536,s3537,movie,15-minute guided meditation,mark knight,anne-marie newland,NaN,NaN,2016,all,special interest,classically trained in the art of yoga in indi...,as3537,19,min
6639,s6640,movie,ink,laurent king,"joseph malerba, julie chevallier, guillaume de...",NaN,NaN,2018,16+,"drama, suspense","an ex para-trooper turned teamster, a small bu...",as6640,69,min
4644,s4645,movie,premonition,gavin wilding,"cynthia preston, adrian paul, christopher lloyd",NaN,NaN,2000,r,suspense,a young journalist is assigned to work with a ...,as4645,94,min
5622,s5623,movie,nazi undead,steven spiel,"georgia chara, leigh scully, andy mcphee",NaN,NaN,2019,18+,horror,a holiday in germany turns into a nightmare wh...,as5623,78,min
4593,s4594,movie,crown heights,matt ruskin,"ron canada, nestor carbonell, natalie paul, am...",united states,NaN,2017,r,drama,when colin warner is wrongfully convicted of m...,as4594,100,min
7911,s7912,movie,out of bounds,richard tuggle,"anthony michael hall, jenny wright, jeff kober...",NaN,NaN,1986,r,"action, drama",daryl's parents ship him off to the big city t...,as7912,93,min


In [6]:
df_disney_plus.sample(20)

,show_id,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,id,duration_int,duration_type
303,s304,movie,you again,andy fickman,"kristen bell, sigourney weaver, jamie lee curt...",united states,2021-02-12,2010,pg,comedy,an unlikely reunion causes high school drama t...,ds304,106,min
684,s685,tv show,one day at disney (shorts),NaN,NaN,NaN,2019-12-06,2019,tv-pg,"anthology, docuseries, family",meet the diverse group of people behind some o...,ds685,1,season
542,s543,tv show,lost treasures of egypt,NaN,rick robles,united kingdom,2020-05-01,2018,tv-pg,"docuseries, historical","an immersive, discovery-led series following t...",ds543,1,season
916,s917,movie,going to the mat,stuart gillard,"andrew lawrence, khleo thomas, alessandra tore...",united states,2019-11-12,2004,tv-g,"coming of age, sports","at a new school, jace overcomes his blindness ...",ds917,92,min
416,s417,movie,d2: the mighty ducks,sam weisman,"emilio estevez, michael tucker, jan rubeš, kat...",united states,2020-09-04,1994,pg,"comedy, drama, family",the mighty ducks face the meanest team on ice ...,ds417,107,min
1017,s1018,movie,life with mikey,james lapine,"michael fox, nathan lane, cyndi lauper, david ...",united states,2019-11-12,1993,pg,"buddy, comedy, coming of age",an ex-child star struggling as an agent discov...,ds1018,92,min
1391,s1392,movie,tuck everlasting,jay russell,"alexis bledel, ben kingsley, sissy spacek, amy...",united states,2019-11-12,2002,pg,"coming of age, drama, fantasy",a woman falls in love with a man who is immortal.,ds1392,91,min
1245,s1246,movie,the african lion,james algar,winston hibler,united states,2019-11-12,1955,g,"animals & nature, documentary, family",two wildlife experts spend years studying the ...,ds1246,72,min
350,s351,movie,the nutcracker and the four realms,"lasse hallström, joe johnston","keira knightley, mackenzie foy, eugenio derbez...",united states,2020-12-04,2018,pg,"action-adventure, family, fantasy",from disney comes the reimagined tale of the n...,ds351,101,min
144,s145,movie,garfield,peter hewitt,"breckin meyer, jennifer love hewitt, stephen t...",united states,2021-07-30,2004,pg,"comedy, family",garfield lures odie out of the house but once ...,ds145,82,min


In [7]:
df_hulu.sample(20)

,show_id,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,id,duration_int,duration_type
2157,s2158,movie,somm: into the bottle,NaN,NaN,united states,2019-07-18,2015,90 min,documentaries,the seldom seen world that surrounds the wine ...,hs2158,0,NaN
1034,s1035,tv show,is it wrong to try to pick up girls in a dungeon?,NaN,NaN,japan,2021-01-17,2015,tv-ma,"action, adventure, anime","in orario, fearless adventurers band together ...",hs1035,3,season
1384,s1385,tv show,if loving you is wrong,NaN,NaN,united states,2020-09-23,2014,tv-ma,"black stories, drama, romance",couples and friends in a middle-class communit...,hs1385,5,season
1608,s1609,tv show,tyler perry's the haves and the have nots,NaN,NaN,NaN,2020-06-10,2013,tv-14,"black stories, drama","in savannah, ga., the cryer family is the envi...",hs1609,7,season
1606,s1607,movie,survival island,NaN,NaN,"united states, japan",2020-06-11,1996,96 min,documentaries,penguins and sea lions migrate to south georgi...,hs1607,0,NaN
2945,s2946,movie,sabrina down under,NaN,NaN,united states,2014-04-25,1999,tv-g,"comedy, family, teen",sabrina aspires to learn about marine biology;...,hs2946,87,min
8,s9,tv show,queens,NaN,NaN,NaN,2021-10-20,2021,tv-14,"drama, music",four women in their 40s reunite for a chance t...,hs9,1,season
758,s759,movie,we were soldiers,NaN,NaN,NaN,2021-05-01,2002,r,"action, adventure, drama",mel gibson stars in this powerful epic chronic...,hs759,139,min
2815,s2816,tv show,jorge el curioso,NaN,NaN,NaN,2016-03-31,2006,tv-g,"adventure, kids, latino","durante casi 65 años, a generaciones de niños ...",hs2816,9,season
1191,s1192,movie,the sunlit night,NaN,NaN,"germany, norway",2020-11-14,2020,82 min,"comedy, romance",the sunlit night follows an aspiring painter f...,hs1192,0,NaN


In [8]:
df_netflix.sample(20)

,show_id,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,id,duration_int,duration_type
7777,s7778,tv show,power rangers super megaforce,NaN,"andrew m. gray, ciara hanna, john mark louderm...",NaN,2016-01-01,2014,tv-y7,kids' tv,armed with a key that allows them to morph int...,ns7778,1,season
4532,s4533,movie,malevolent,olaf de fleur,"florence pugh, ben lloyd-hughes, celia imrie, ...",united kingdom,2018-10-05,2018,tv-ma,"horror movies, thrillers",a brother-sister team who fake paranormal enco...,ns4533,89,min
429,s430,movie,sanitation day,seyi babatope,"blossom chukwujekwu, elozonam ogbolu, nse ikpe...",nigeria,2021-07-21,2020,tv-14,"dramas, international movies, thrillers",two cops must contend with the uncooperative t...,ns430,105,min
4424,s4425,movie,arjun: the warrior prince,arnab chaudhuri,"yudhveer bakoliya, ashok banthia, ravi khanwil...",india,2018-11-01,2011,tv-14,"action & adventure, dramas, international movies",this animated retelling of the classic indian ...,ns4425,91,min
4596,s4597,movie,peace haven,suman ghosh,"soumitra chatterjee, paran banerjee, arun mukh...","india, united states",2018-10-01,2016,tv-14,"comedies, dramas, independent movies",complications around the cremation of their de...,ns4597,73,min
5035,s5036,tv show,dragons: race to the edge,NaN,"jay baruchel, america ferrera, t.j. miller, ch...",united states,2018-02-16,2018,tv-y7,"kids' tv, tv action & adventure","from the creators of ""how to train your dragon...",ns5036,6,season
7880,s7881,movie,rocky ii,sylvester stallone,"sylvester stallone, talia shire, burt young, c...",united states,2019-08-01,1979,pg,"dramas, sports movies","featuring a rousing climax, this engaging sequ...",ns7881,119,min
8615,s8616,movie,traitor,jeffrey nachmanoff,"don cheadle, guy pearce, saïd taghmaoui, neal ...",united states,2018-07-01,2008,pg-13,"dramas, thrillers",an fbi agent assigned to pursue an ex-special ...,ns8616,114,min
5901,s5902,movie,tig,"kristina goolsby, ashley york",tig notaro,united states,2015-07-17,2015,tv-14,"documentaries, stand-up comedy",comedian tig notaro announced her cancer diagn...,ns5902,92,min
8580,s8581,movie,thor: ragnarok,taika waititi,"chris hemsworth, tom hiddleston, cate blanchet...",united states,2018-06-05,2017,pg-13,"action & adventure, comedies, sci-fi & fantasy",to save asgard from a bloodthirsty goddess of ...,ns8581,131,min


In [9]:
df_ratings.sample(20)

,userid,scored,timestamp,movieid,scoreds
8192460,256663,2.5,2008-02-26,ns4668,3.6
6247517,64461,2.5,2010-07-07,as1123,3.5
3413363,35501,3.5,2006-06-07,ns2570,3.6
8965174,265097,2.0,1997-05-30,hs1125,3.6
7045326,72675,2.0,2000-11-20,as5762,3.5
1266145,12979,5.0,2012-09-18,hs2373,3.6
3359173,34949,3.5,2005-04-03,ns28,3.4
10711969,121199,2.5,2008-01-11,ns7985,3.5
4265079,44246,4.5,2008-09-17,ns7199,3.5
5741260,59156,3.5,2008-07-28,as3109,3.6


Observamos la informacion detallada de cada dataframe

In [10]:
df_amazon_prime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   show_id        9668 non-null   object
 1   type           9668 non-null   object
 2   title          9668 non-null   object
 3   director       7586 non-null   object
 4   cast           8435 non-null   object
 5   country        672 non-null    object
 6   date_added     155 non-null    object
 7   release_year   9668 non-null   int64 
 8   rating         9668 non-null   object
 9   listed_in      9668 non-null   object
 10  description    9668 non-null   object
 11  id             9668 non-null   object
 12  duration_int   9668 non-null   int64 
 13  duration_type  9668 non-null   object
dtypes: int64(2), object(12)
memory usage: 1.0+ MB


In [11]:
df_disney_plus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   show_id        1450 non-null   object
 1   type           1450 non-null   object
 2   title          1450 non-null   object
 3   director       977 non-null    object
 4   cast           1260 non-null   object
 5   country        1231 non-null   object
 6   date_added     1447 non-null   object
 7   release_year   1450 non-null   int64 
 8   rating         1450 non-null   object
 9   listed_in      1450 non-null   object
 10  description    1450 non-null   object
 11  id             1450 non-null   object
 12  duration_int   1450 non-null   int64 
 13  duration_type  1450 non-null   object
dtypes: int64(2), object(12)
memory usage: 158.7+ KB


In [12]:
df_hulu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3073 entries, 0 to 3072
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   show_id        3073 non-null   object 
 1   type           3073 non-null   object 
 2   title          3073 non-null   object 
 3   director       3 non-null      object 
 4   cast           0 non-null      float64
 5   country        1620 non-null   object 
 6   date_added     3045 non-null   object 
 7   release_year   3073 non-null   int64  
 8   rating         3073 non-null   object 
 9   listed_in      3073 non-null   object 
 10  description    3069 non-null   object 
 11  id             3073 non-null   object 
 12  duration_int   3073 non-null   int64  
 13  duration_type  2594 non-null   object 
dtypes: float64(1), int64(2), object(11)
memory usage: 336.2+ KB


In [13]:
df_netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   show_id        8807 non-null   object
 1   type           8807 non-null   object
 2   title          8807 non-null   object
 3   director       6173 non-null   object
 4   cast           7982 non-null   object
 5   country        7976 non-null   object
 6   date_added     8797 non-null   object
 7   release_year   8807 non-null   int64 
 8   rating         8807 non-null   object
 9   listed_in      8807 non-null   object
 10  description    8807 non-null   object
 11  id             8807 non-null   object
 12  duration_int   8807 non-null   int64 
 13  duration_type  8804 non-null   object
dtypes: int64(2), object(12)
memory usage: 963.4+ KB


In [15]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11024289 entries, 0 to 11024288
Data columns (total 5 columns):
 #   Column     Dtype         
---  ------     -----         
 0   userid     int64         
 1   scored     float64       
 2   timestamp  datetime64[ns]
 3   movieid    object        
 4   scoreds    float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 420.5+ MB


Se analizaran los valores faltantes

In [10]:
df_amazon_prime.isnull().sum()

show_id             0
type                0
title               0
director         2082
cast             1233
country          8996
date_added       9513
release_year        0
rating              0
listed_in           0
description         0
id                  0
duration_int        0
duration_type       0
dtype: int64

In [12]:
df_disney_plus.isnull().sum()

show_id            0
type               0
title              0
director         473
cast             190
country          219
date_added         3
release_year       0
rating             0
listed_in          0
description        0
id                 0
duration_int       0
duration_type      0
dtype: int64

In [11]:
df_hulu.isnull().sum()

show_id             0
type                0
title               0
director         3070
cast             3073
country          1453
date_added         28
release_year        0
rating              0
listed_in           0
description         4
id                  0
duration_int        0
duration_type     479
dtype: int64

In [13]:
df_netflix.isnull().sum()

show_id             0
type                0
title               0
director         2634
cast              825
country           831
date_added         10
release_year        0
rating              0
listed_in           0
description         0
id                  0
duration_int        0
duration_type       3
dtype: int64

In [14]:
df_ratings.isnull().sum()

userid       0
scored       0
timestamp    0
movieid      0
scoreds      0
dtype: int64

**Se analizaran las columnas con valores numericos**

Como a la columna 'date_added' no lo toma con formato DATETIME, se convierte.

In [14]:
df_amazon_prime['date_added'] = pd.to_datetime(df_amazon_prime['date_added'])
df_disney_plus['date_added'] = pd.to_datetime(df_disney_plus['date_added'])
df_hulu['date_added'] = pd.to_datetime(df_hulu['date_added'])
df_netflix['date_added'] = pd.to_datetime(df_netflix['date_added'])

In [12]:
df_amazon_prime['date_added'].min()

Timestamp('2021-03-30 00:00:00')

In [15]:
df_disney_plus['date_added'].min()

Timestamp('2019-10-01 00:00:00')

In [16]:
df_hulu['date_added'].min()

Timestamp('2006-07-14 00:00:00')

In [6]:
df_ratings['timestamp'].min()

Timestamp('1995-01-09 00:00:00')

**Se analizaran si hay duplicados**

In [4]:
df_ratings.duplicated().value_counts()

False    11013823
True        10466
dtype: int64

**Se identificaran valores atípicos en la columna "scored" del dataframe ratings**

In [8]:
q1 = df_ratings['scored'].quantile(0.25)
q3 = df_ratings['scored'].quantile(0.75)
iqr = q3 - q1

# define los límites para los outliers
lower_limit = q1 - 1.5 * iqr
upper_limit = q3 + 1.5 * iqr

# identifica los outliers
outliers = df_ratings[(df_ratings['scored'] < lower_limit) | (df_ratings['scored'] > upper_limit)]

# muestra los outliers
print(outliers)

          userid  scored  timestamp movieid  scoreds
0              1     1.0 2015-03-09   as680      3.5
24             1     0.5 2015-03-09  as7193      3.5
42             2     1.0 1997-06-23  hs1914      3.6
43             2     1.0 1997-06-23  as9455      3.5
84             4     1.0 2003-01-15  ns3279      3.6
...          ...     ...        ...     ...      ...
11024167  124373     1.0 2001-05-31  ns7039      3.5
11024171  124374     1.0 2001-11-28  ds1361      3.6
11024179  124374     1.0 2001-11-28  as9146      3.6
11024192  124375     1.0 1996-08-14   hs282      3.4
11024218  124376     1.0 2001-04-26  ns5214      3.6

[514764 rows x 5 columns]


El resultado que te devolvió es un subconjunto de datos de la tabla. En este subconjunto, se han eliminado todas las filas donde los valores de la columna "scored" se encuentran fuera del rango intercuartil, lo que sugiere que son posibles valores atípicos o "outliers". En otras palabras, estos son los datos que no son atípicos en la columna "scored". La tabla resultante tiene 514764 filas y 5 columnas. Cada fila contiene información de un usuario que ha calificado una película, incluyendo el ID del usuario, su calificación ("scored"), la fecha en que se hizo la calificación ("timestamp"), el ID de la película ("movieid") y el promedio de todas las calificaciones para esa película ("scored").

**CONCLUSIONES EDA**

GENERALES (considerando el objetivo y alcance de este proyecto):

- Las columnas duration tienen tanto información en minutos como de temporadas

- Las columnas cast tienen los actores separados por una coma

PARTICULARES

1- df_amazon_prime_ETL

- separdor ','+ encoding utf-8
- cararcteres especiales [ (30)
- valores faltantes
    + director   2082
    + cast       1233
    + country    8996
    + date_added 9513
- 9687 registros
- 12 columnas

2- df_disney_plus_ETL

- separdor ','+ encoding utf-8
- cararcteres especiales [ (NO)
- valores faltantes
    + director      473
    + cast          190
    + country       219
    + date_added    3
- 1450 registros
- 12 columnas


3- df_hulu_ETL

- separdor ','+ encoding utf-8
- cararcteres especiales [ (1)
- valores faltantes
    + director      3070
    + cast          3073
    + country       1453
    + date_added    28
    + description   4
    + duration_type 479
- 3090 registros
- 12 columnas

4- df_netflix_ETL

- valores faltantes
    + director 2634
    + cast 825
    + country 831
    + date_added 10
    + duration 3
- 8807 registros
- 12 columnas

5- df_rating_ETL
-valores faltantes 0
- 11024289 registros
- 5 columnas
- duplicados 10466

Al obtener el primer año de ingreso de la columna 'timestamp' (1995-01-09),se concluyo que es un valor fuera de lo normal dado que las plataformas abrieron años depues. 